
# MNIST classification with CNN

Inspired by the [CNN MNIST Fidle tutorial](https://fidle.cnrs.fr/w3/)

TODO :
- [ ] Train
- [ ] Save model
- [ ] Evaluate (with some fancy plots)
- [ ] Migrate on GPU
- [ ] Make predictions
- [ ] ...

## 0. Parameters

In [1]:
batch_size = 512
num_epochs = 16

## 1. Imports

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time as time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

## 2. Retrieve and prepare the data

In [3]:
# Load the data
transform = transforms.Compose([
	transforms.ToTensor(),
	transforms.Normalize((0.5,), (0.5,))
])

trainset = MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=4, shuffle=True)

testset = MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=4, shuffle=False)

# Verify the shape of the data
dataiter = iter(trainloader)
images, labels = next(dataiter)
print(images.shape)
print(labels.shape)

dataiter = iter(testloader)
images, labels = next(dataiter)
print(images.shape)
print(labels.shape)

Using downloaded and verified file: ./data/MNIST/raw/train-images-idx3-ubyte.gz
Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Using downloaded and verified file: ./data/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

torch.Size([4, 1, 28, 28])
torch.Size([4])
torch.Size([4, 1, 28, 28])
torch.Size([4])


## 3. Create the model

In [4]:
# Define the model
class Model(nn.Module):
	def __init__(self):
		super(Model, self).__init__()

		self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
		self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
		self.dropout1 = nn.Dropout(0.2)

		self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
		self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
		self.dropout2 = nn.Dropout(0.2)

		self.fc1 = nn.Linear(16 * 7 * 7, 100)
		self.dropout3 = nn.Dropout(0.5)
		self.fc2 = nn.Linear(100, 10)

	def forward(self, x):
		x = self.pool1(F.relu(self.conv1(x)))
		x = self.dropout1(x)
		x = self.pool2(F.relu(self.conv2(x)))
		x = self.dropout2(x)
		x = x.view(-1, 16 * 7 * 7) # Flatten
		x = F.relu(self.fc1(x))
		x = self.dropout3(x)
		x = self.fc2(x)
		return x

# Instantiate the model
model = Model()
display(model)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Model(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.2, inplace=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

## 5. Train the model

In [ ]:
batch_size  = 512
epochs      =  16

history = model.fit(  x_train, y_train,
                      batch_size      = batch_size,
                      epochs          = epochs,
                      verbose         = fit_verbosity,
                      validation_data = (x_test, y_test))

## Step 6 - Evaluate
### 6.1 - Final loss and accuracy
Note : With a DNN, we had a precision of the order of : 97.7%

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)

print(f'Test loss     : {score[0]:4.4f}')
print(f'Test accuracy : {score[1]:4.4f}')

### 6.2 - Plot history

In [ ]:
fidle.scrawler.history(history, figsize=(6,4), save_as='03-history')

### 6.3 - Plot results

In [ ]:
#y_pred   = model.predict_classes(x_test)           Deprecated after 01/01/2021 !!

y_sigmoid = model.predict(x_test, verbose=fit_verbosity)
y_pred    = np.argmax(y_sigmoid, axis=-1)

fidle.scrawler.images(x_test, y_test, range(0,200), columns=12, x_size=1, y_size=1, y_pred=y_pred, save_as='04-predictions')

### 6.4 - Plot some errors

In [ ]:
errors=[ i for i in range(len(x_test)) if y_pred[i]!=y_test[i] ]
errors=errors[:min(24,len(errors))]
fidle.scrawler.images(x_test, y_test, errors[:15], columns=6, x_size=2, y_size=2, y_pred=y_pred, save_as='05-some-errors')

In [ ]:
fidle.scrawler.confusion_matrix(y_test,y_pred,range(10),normalize=True, save_as='06-confusion-matrix')

In [ ]:
fidle.end()

<div class="todo">
    A few things you can do for fun:
    <ul>
        <li>Changing the network architecture (layers, number of neurons, etc.)</li>
        <li>Display a summary of the network</li>
        <li>Retrieve and display the softmax output of the network, to evaluate its "doubts".</li>
    </ul>
</div>

---
<img width="80px" src="../fidle/img/logo-paysage.svg"></img>